# <h2>Importación de librerias</h2>

In [1]:
# Visualization Libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Operational Libraries
import pandas as pd
import re
import dask.dataframe as dd
import numpy as np

# <h2>1. Carga de datos desde excel a dataframes de pandas</h2>


En primer lugar voy a cargar cada uno de los csv a unos dataframes de pandas para ir manejandolos desde aqui. La idea es ir cruzando algunos excels y ver qué graficos interesantes van saliendo de los datos que tenemos. Los voy a cargar todos de una vez y a continuación los veremos en tablas para ir describiendolos

Esto puede tomar un ratito

In [2]:
df_tratamiento = pd.read_csv('Tratamiento.csv')

In [3]:
df_informacion_de_clientes = pd.read_csv('Informacion_Clientes.csv')

In [4]:
df_simulaciones_clientes = pd.read_csv('Simulaciones_Clientes.csv')

In [5]:
df_ventas = pd.read_csv('Ventas.csv')

In [6]:
df_simulaciones_clientes[df_simulaciones_clientes['Monto_Simulado'] < 0].count()
df_simulaciones_clientes = df_simulaciones_clientes[df_simulaciones_clientes['Monto_Simulado'] > 0]


<h3>1.1 Carga de 'Informacion_Clientes.csv'</h3>

En primer lugar se cargará la información de los clientes. Esto incluye las siguientes características de los clientes:

* **unnamed**: algo como uid
* **Rut**: identificador de Chile (supongo que por privacidad va desde 0 a max de observaciones)
* **Género**: Masculino o femenino
* **Categoría_Digital**: Si el cliente es digital o no
* **Elasticidad_Precios**: Baja, media o alta
* **Nacionalidad**: Chileno o extranjero
* **Propensión**: Número entre 0 y 1 que idica que tan propenso a cursar un credito es el cliente
* **Probabilidad_No_Pago**: Número entre 0 y 1 que indica la probabilidad de que el cliente no pague la deuda
* **Edad**: Numero entero de edad en años
* **Renta**: Renta promedio de los últimos 12 meses
* **Oferta_Consumo**: Monto máximo que puede cursar un cliente dado sus antecedentes crediticios y situación socioeconómica. 
* **Deuda_CMF**: Deuda que tiene el cliente en otros bancos. Efectivamente es deuda pendiente, pero de créditos otorgados por la competencia.
* **Tiempo_como_cliente**: Número de tiempo(no sé en que medida está) que el cliente lleva en el banco


<span style="color:cyan">Algo interesante es que tiempo como cliente es solamente '2'</span>

In [7]:
df_informacion_de_clientes.drop(columns=['Tiempo_como_cliente'], inplace=True)

<h3>1.2 Carga de 'Simulaciones_Clientes.csv'</h3>

En segundo lugar se cargaran las simulaciones hechas por los clientes en la página del banco. Esto incluye las siguientes características de las simulaciones:
* **unnamed**: Supongo que es el número de simulacion registrada, un tipo de identificador de la simulación
* **fecha**: yyyy-mm-dd fecha de la simulación
* **rut**: identificador de Chile del cliente que hizo la simulacion
* **monto_simulado**: monto prestado al cliente
* **plazo_simulado**: plazo en **meses** del crédito
* **tasa_simulado**: costo para el cliente del credito

Lo interesante es que con el rut se pueden cruzar el dataframe de informacion de cliente y del plazo simulado

<h3>1.3 Carga de 'Tratamiento.csv'</h3>

En tercer lugar se cargara el tratamiento que ha tenido el banco con el cliente, es decir, cómo se han contactado con él. Esto incluye las siguientes características:

* **unnamed**: Número de tratamiento registrado
* **fecha**: yyyy-mm-dd
* **rut**: Identificador de Chile del cliente con el que se tiene el tipo de trato
* **n_correos**: Cantidad de correos que se enviaron en el mes que sale la fecha. Es decir, si sele fecha '2024-03-01', correspondería a los correos enviados en marzo de 2024.
* **asg_ejec**: Si el cliente tiene un ejecutivo asignado

-[Notas de jp]: Este es el que más me costo entender, está registrado el tratamiento por fecha. Hay muchos más tratamientos(3 millones y medio) que clientes(500 mil clientes) por lo que supongo que el tipo de trato que le dan al cliente pudo haber cambiado en el tiempo. No estoy seguro si la cantidad de correos es un recuento del total de correos hasta la fecha o cuantos se les envió esa fecha.

<h3>1.4 Carga de 'Ventas.csv'</h3>

Por último se cargaran las ventas que ha tenido el banco con el cliente. Esto incluye las siguientes características:

* **unnamed**: Índice sin significado
* **fecha**: yyyy-mm-dd -> fecha en la que se concretó la venta
* **rut**: identificador de Chile del cliente al que se le concretó la venta

# <h2>2. Joints de datos<h2>


Ya con todos los df cargados se pueden ir cruzando algunos de ellos para poder ir sacando algunas conclusiones.

El primer joint que haré es la informacion de los clientes y la simulacion. 

Quiero saber: de todos mis clientes quienes son los que simulan. Por lo tanto, para cada uno de los clientes que exista en la base de informacion de clientes quiero tener la informacion de sus simulaciones (o no simulaciones es que no han simulado). Esto es interesante para saber si simulan o no.

In [8]:
df_simulaciones_e_informacion_de_clientes = pd.merge(
    df_informacion_de_clientes, 
    df_simulaciones_clientes, 
    on='rut', 
    how='left')

Revisaremos si hay algun NaN en este merge, es decir, si algún cliente no ha simulado.

In [9]:
df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].isna().sum()

747

In [10]:
df_simulaciones_e_informacion_de_clientes[df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].isna()]['rut'].nunique()

747

<span style="color:cyan">Hay 747 personas que nunca simularon.</span>

Esto lo agregaré como columna llamada simulo, que es binaria e indica si el cliente simuló o no.


In [11]:
df_simulaciones_e_informacion_de_clientes['simulo'] = df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].notna().astype(int)

In [12]:
df_simulaciones_e_informacion_de_clientes.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)

In [13]:
df_simulaciones_e_informacion_de_clientes_ventas = pd.merge( 
    df_simulaciones_e_informacion_de_clientes, 
    df_ventas, 
    on=['rut', 'fecha'], 
    how='left'
    )

# 3. Procesamiento de tablas para ilan [ 1 ]
Tabla 1 (Ganancias): Necesito una tabla con todas las ventas, que tenga:

- fecha
- ⁠id cliente
- ⁠probabilidad no pago de ese cliente
- ⁠tasa
- ⁠monto
- ⁠plazo

In [14]:
# df_simulaciones_e_informacion_de_clientes_ventas

In [15]:
# df_simulaciones_e_informacion_de_clientes_filtered = df_simulaciones_e_informacion_de_clientes_ventas[df_simulaciones_e_informacion_de_clientes_ventas['Unnamed: 0'].notna()]
# tabla_ganancias = df_simulaciones_e_informacion_de_clientes_filtered[['rut','Probabilidad_No_Pago','Tasa_Simulado','fecha','Monto_Simulado','Plazo_Simulado']].copy()

# tabla_ganancias

# 3.2 Procesamiento de tablas para ilan [ 2 ]
Tabla 2 (créditos y correos): Necesito que me envíes por mes una tabla con:

- id cliente
- ⁠mes
- ⁠créditos simulados
- ⁠créditos aprobados
- ⁠correos enviados
- asignacion de ejecutivo

In [16]:
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento = pd.merge( 
    df_simulaciones_e_informacion_de_clientes_ventas, 
    df_tratamiento, 
    on=['rut', 'fecha'], 
    how='left'
    )

In [17]:
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.rename(columns={'Unnamed: 0_x': 'venta'}, inplace=True)
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'] = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'].notna().astype(int)
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,fecha,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo,venta,Unnamed: 0_y,n_correos,asg_ejec
0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-10-01,319936.0,33.0,1.092295,1,0,0.0,2.0,0.0
1,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-11-01,249773.0,30.0,2.324675,1,0,0.0,0.0,0.0
2,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2020-02-01,280087.0,28.0,1.051704,1,1,0.0,1.0,0.0
3,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-05-01,289780.0,26.0,2.193118,1,0,0.0,1.0,0.0
4,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-06-01,258061.0,22.0,2.188368,1,1,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517080,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-03-01,403331.0,17.0,2.356028,1,0,543650.0,2.0,1.0
8517081,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-08-01,359897.0,16.0,2.478376,1,0,543650.0,2.0,0.0
8517082,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-03-01,348048.0,18.0,2.301079,1,0,543650.0,4.0,0.0
8517083,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-05-01,344504.0,16.0,2.462272,1,1,543650.0,0.0,1.0


In [18]:
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['mes'] = pd.to_datetime(df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha']).dt.to_period('M')

tabla_creditos_correos = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.groupby(['rut', 'mes']).agg({
    'Monto_Simulado': 'count',
    'venta': 'sum',
    'n_correos': 'sum',
    'asg_ejec': 'max'
}).reset_index()

tabla_creditos_correos.rename(columns={
    'rut': 'id cliente',
    'Monto_Simulado': 'créditos simulados',
    'venta': 'créditos aprobados',
    'n_correos': 'correos enviados',
    'asg_ejec': 'asignacion de ejecutivo'
}, inplace=True)

tabla_creditos_correos

,id cliente,mes,créditos simulados,créditos aprobados,correos enviados,asignacion de ejecutivo
0,1,2019-10,1,0,2.0,0.0
1,1,2019-11,1,0,0.0,0.0
2,1,2020-02,1,1,1.0,0.0
3,1,2021-05,1,0,1.0,0.0
4,1,2021-06,1,1,3.0,0.0
...,...,...,...,...,...,...
8516333,543651,2023-03,1,0,2.0,1.0
8516334,543651,2023-08,1,0,2.0,0.0
8516335,543651,2024-03,1,0,4.0,0.0
8516336,543651,2024-05,1,1,0.0,1.0


# 3.5 Tratamiento que no simuló

In [19]:
df_clientes_que_no_simulan = df_simulaciones_e_informacion_de_clientes[df_simulaciones_e_informacion_de_clientes['simulo'] == 0]
df_clientes_que_no_simulan

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,fecha,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo
12137,780,Femenino,Cliente Digital,Alta,Chileno,0.307640,0.007871,31.0,4.744469e+05,1117352.0,4.317737e+05,NaN,NaN,NaN,NaN,0
32403,2082,Femenino,Cliente Digital,Media,Chileno,0.505345,0.025828,27.0,7.642129e+05,2078877.0,9.941304e+05,NaN,NaN,NaN,NaN,0
53352,3403,Masculino,Cliente Digital,Baja,Chileno,0.860546,0.013709,28.0,5.206620e+04,110955.0,6.067930e+04,NaN,NaN,NaN,NaN,0
80728,5150,Femenino,Cliente Digital,Media,Chileno,0.375278,0.029899,29.0,4.578795e+06,9640006.0,5.446584e+06,NaN,NaN,NaN,NaN,0
94914,6060,Masculino,Cliente Digital,Alta,Chileno,0.422165,0.053365,27.0,7.495721e+05,1953276.0,8.010537e+05,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8475532,540942,Femenino,Cliente no Digital,Alta,Extranjero,0.145846,0.015207,29.0,6.282168e+06,17959862.0,7.883997e+06,NaN,NaN,NaN,NaN,0
8490307,541897,Masculino,Cliente no Digital,Baja,Chileno,0.433438,0.010885,47.0,7.916352e+06,10614996.0,7.610865e+06,NaN,NaN,NaN,NaN,0
8495217,542225,Femenino,Cliente Digital,Alta,Chileno,0.511740,0.030114,31.0,8.167334e+05,3469995.0,7.542526e+05,NaN,NaN,NaN,NaN,0
8496664,542322,Masculino,Cliente no Digital,Alta,Chileno,0.828991,0.029804,29.0,9.794209e+05,3549490.0,8.051102e+05,NaN,NaN,NaN,NaN,0


In [20]:
df_clientes_que_no_simulan.describe()

,rut,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo
count,747.000000,747.000000,747.000000,747.000000,7.470000e+02,7.470000e+02,7.470000e+02,0.0,0.0,0.0,747.0
mean,279735.147256,0.502508,0.020988,31.068273,1.726504e+06,6.700486e+06,1.745132e+06,NaN,NaN,NaN,0.0
std,158062.373498,0.284599,0.015538,5.179812,4.942094e+06,2.882054e+07,5.062681e+06,NaN,NaN,NaN,0.0
min,780.000000,0.000093,0.000006,25.000000,1.123287e+03,2.214000e+03,5.646986e+02,NaN,NaN,NaN,0.0
25%,147122.000000,0.271560,0.008383,29.000000,3.073471e+05,7.936645e+05,2.856043e+05,NaN,NaN,NaN,0.0
50%,281422.000000,0.494136,0.018703,30.000000,7.184987e+05,1.909540e+06,6.908328e+05,NaN,NaN,NaN,0.0
75%,419937.500000,0.750950,0.029896,32.000000,1.499324e+06,4.196426e+06,1.457513e+06,NaN,NaN,NaN,0.0
max,543624.000000,0.997085,0.069568,63.000000,6.620296e+07,4.499284e+08,6.043560e+07,NaN,NaN,NaN,0.0


In [21]:
df_clientes_que_no_simulan_tratamiento = pd.merge(df_clientes_que_no_simulan, df_tratamiento, on=['rut'], how='left')

In [22]:
df_clientes_que_no_simulan_tratamiento.describe()

,rut,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo,Unnamed: 0,n_correos,asg_ejec
count,49302.000000,49302.000000,49302.000000,49302.000000,4.930200e+04,4.930200e+04,4.930200e+04,0.0,0.0,0.0,49302.0,49302.000000,49302.000000,49302.000000
mean,279735.147256,0.502508,0.020988,31.068273,1.726504e+06,6.700486e+06,1.745132e+06,NaN,NaN,NaN,0.0,279734.147256,2.317959,0.340980
std,157958.141911,0.284411,0.015528,5.176396,4.938835e+06,2.880153e+07,5.059343e+06,NaN,NaN,NaN,0.0,157958.141911,1.319536,0.474043
min,780.000000,0.000093,0.000006,25.000000,1.123287e+03,2.214000e+03,5.646986e+02,NaN,NaN,NaN,0.0,779.000000,0.000000,0.000000
25%,145851.000000,0.271351,0.008380,29.000000,3.072792e+05,7.933310e+05,2.828713e+05,NaN,NaN,NaN,0.0,145850.000000,1.000000,0.000000
50%,281422.000000,0.494136,0.018703,30.000000,7.184987e+05,1.909540e+06,6.908328e+05,NaN,NaN,NaN,0.0,281421.000000,2.000000,0.000000
75%,419985.000000,0.752659,0.029899,32.000000,1.499854e+06,4.199411e+06,1.462403e+06,NaN,NaN,NaN,0.0,419984.000000,4.000000,1.000000
max,543624.000000,0.997085,0.069568,63.000000,6.620296e+07,4.499284e+08,6.043560e+07,NaN,NaN,NaN,0.0,543623.000000,4.000000,1.000000


# 6. Clustering

In [23]:
df_simulaciones_e_informacion_de_clientes_ventas['venta'] = df_simulaciones_e_informacion_de_clientes_ventas['Unnamed: 0'].notna().astype(int)

In [24]:
df_simulaciones_e_informacion_de_clientes_ventas.drop(columns=['Unnamed: 0'], inplace=True)

In [25]:
datos_deventas_y_simulaciones = df_simulaciones_e_informacion_de_clientes_ventas.groupby('rut').agg({'simulo': 'sum', 'venta': 'sum'})

In [26]:
datos_deventas_y_simulaciones.reset_index(drop=True, inplace=True)


In [27]:
df_informacion_de_clientes['cantidad simulaciones'] = datos_deventas_y_simulaciones['simulo']
df_informacion_de_clientes['cantidad de ventas'] = datos_deventas_y_simulaciones['venta']
df_informacion_de_clientes['tasa de concresion'] = df_informacion_de_clientes['cantidad de ventas'] / df_informacion_de_clientes['cantidad simulaciones']

In [28]:
df_informacion_de_clientes['tasa de concresion'].fillna(0, inplace=True)

In [29]:
df_informacion_de_clientes_mapped = df_informacion_de_clientes.copy()

df_informacion_de_clientes_mapped['Genero'] = df_informacion_de_clientes['Genero'].map({'Masculino': 1, 'Femenino': 0})
df_informacion_de_clientes_mapped['Categoria_Digital'] = df_informacion_de_clientes['Categoria_Digital'].map({'Cliente Digital': 1, 'Cliente no Digital': 0})
df_informacion_de_clientes_mapped['Elasticidad_Precios'] = df_informacion_de_clientes['Elasticidad_Precios'].map({'Alta': 3, 'Media': 2, 'Baja': 1})
df_informacion_de_clientes_mapped['Nacionalidad'] = df_informacion_de_clientes['Nacionalidad'].map({'Chileno': 1, 'Extranjero': 0})

df_informacion_de_clientes_mapped


,Unnamed: 0,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,cantidad simulaciones,cantidad de ventas,tasa de concresion
0,0,1,1,0,3,1,0.997340,0.028445,30.0,6.258183e+05,2164276.0,7.125854e+05,15,5,0.333333
1,1,2,0,0,1,1,0.291601,0.014320,41.0,3.172616e+05,1451704.0,1.970697e+05,10,2,0.200000
2,2,3,0,0,1,1,0.685085,0.002156,38.0,1.240551e+07,39322607.0,1.400604e+07,20,4,0.200000
3,3,4,1,0,3,1,0.914672,0.034418,57.0,5.441466e+05,1607042.0,5.159465e+05,25,9,0.360000
4,4,5,1,1,2,1,0.425077,0.014978,26.0,1.870225e+05,493865.0,2.601013e+05,17,6,0.352941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543646,543647,0,1,1,1,0.144657,0.037291,29.0,1.176598e+05,9625991.0,1.299826e+05,8,0,0.000000
543647,543647,543648,1,0,1,1,0.740170,0.035877,31.0,1.558612e+06,7522126.0,1.740917e+06,18,3,0.166667
543648,543648,543649,1,0,2,1,0.255285,0.023306,49.0,9.449508e+05,1889172.0,1.057952e+06,14,8,0.571429
543649,543649,543650,0,0,2,1,0.709086,0.015121,40.0,1.039964e+06,4518498.0,1.010872e+06,22,12,0.545455


A continuación voy a normalizar todas las variables. para correr el clustering


In [30]:
df_informacion_clientes_clusterizar = df_informacion_de_clientes_mapped.drop(columns=['Unnamed: 0', 'Oferta_Consumo', 'Deuda_CMF'])

In [31]:
df_informacion_clientes_clusterizar

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,cantidad simulaciones,cantidad de ventas,tasa de concresion
0,1,1,0,3,1,0.997340,0.028445,30.0,6.258183e+05,15,5,0.333333
1,2,0,0,1,1,0.291601,0.014320,41.0,3.172616e+05,10,2,0.200000
2,3,0,0,1,1,0.685085,0.002156,38.0,1.240551e+07,20,4,0.200000
3,4,1,0,3,1,0.914672,0.034418,57.0,5.441466e+05,25,9,0.360000
4,5,1,1,2,1,0.425077,0.014978,26.0,1.870225e+05,17,6,0.352941
...,...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,0,1,1,1,0.144657,0.037291,29.0,1.176598e+05,8,0,0.000000
543647,543648,1,0,1,1,0.740170,0.035877,31.0,1.558612e+06,18,3,0.166667
543648,543649,1,0,2,1,0.255285,0.023306,49.0,9.449508e+05,14,8,0.571429
543649,543650,0,0,2,1,0.709086,0.015121,40.0,1.039964e+06,22,12,0.545455


# 9 Gráficos varios

In [32]:
df_simulaciones_e_informacion_de_clientes_ventas['venta'].fillna(0, inplace=True)

In [33]:
df_simulaciones_e_informacion_de_clientes_ventas

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,Deuda_CMF,fecha,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo,venta
0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-10-01,319936.0,33.0,1.092295,1,0
1,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2019-11-01,249773.0,30.0,2.324675,1,0
2,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2020-02-01,280087.0,28.0,1.051704,1,1
3,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-05-01,289780.0,26.0,2.193118,1,0
4,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,712585.357842,2021-06-01,258061.0,22.0,2.188368,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517080,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-03-01,403331.0,17.0,2.356028,1,0
8517081,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2023-08-01,359897.0,16.0,2.478376,1,0
8517082,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-03-01,348048.0,18.0,2.301079,1,0
8517083,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,574575.649505,2024-05-01,344504.0,16.0,2.462272,1,1


In [34]:
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['n_correos'].isna().sum()
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['n_correos'].fillna(0, inplace=True)
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['asg_ejec'].isna().sum()
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['asg_ejec'].fillna(0, inplace=True)

In [35]:
# Use vectorized string operations to create the 'Tratamiento' column
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['Tratamiento'] = (
    'Ejecutivo=' + df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['asg_ejec'].astype(str) +
    ', Correos=' + df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['n_correos'].astype(int).astype(str)
)

df_simulaciones_e_informacion_de_clientes_ventas_tratamiento

,rut,Genero,Categoria_Digital,Elasticidad_Precios,Nacionalidad,Propension,Probabilidad_No_Pago,Edad,Renta,Oferta_Consumo,...,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,simulo,venta,Unnamed: 0_y,n_correos,asg_ejec,mes,Tratamiento
0,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,...,319936.0,33.0,1.092295,1,0,0.0,2.0,0.0,2019-10,"Ejecutivo=0.0, Correos=2"
1,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,...,249773.0,30.0,2.324675,1,0,0.0,0.0,0.0,2019-11,"Ejecutivo=0.0, Correos=0"
2,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,...,280087.0,28.0,1.051704,1,1,0.0,1.0,0.0,2020-02,"Ejecutivo=0.0, Correos=1"
3,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,...,289780.0,26.0,2.193118,1,0,0.0,1.0,0.0,2021-05,"Ejecutivo=0.0, Correos=1"
4,1,Masculino,Cliente no Digital,Alta,Chileno,0.997340,0.028445,30.0,625818.326221,2164276.0,...,258061.0,22.0,2.188368,1,1,0.0,3.0,0.0,2021-06,"Ejecutivo=0.0, Correos=3"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8517080,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,...,403331.0,17.0,2.356028,1,0,543650.0,2.0,1.0,2023-03,"Ejecutivo=1.0, Correos=2"
8517081,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,...,359897.0,16.0,2.478376,1,0,543650.0,2.0,0.0,2023-08,"Ejecutivo=0.0, Correos=2"
8517082,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,...,348048.0,18.0,2.301079,1,0,543650.0,4.0,0.0,2024-03,"Ejecutivo=0.0, Correos=4"
8517083,543651,Femenino,Cliente no Digital,Media,Chileno,0.860781,0.019647,35.0,472806.728024,1979540.0,...,344504.0,16.0,2.462272,1,1,543650.0,0.0,1.0,2024-05,"Ejecutivo=1.0, Correos=0"


In [36]:
cluster = 'Categoria_Digital'

# Ensure the 'fecha' column is in datetime format
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha'] = pd.to_datetime(df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha'])

# Extract the month from the 'fecha' column
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['Mes'] = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha'].dt.to_period('M')

# Group by 'Cluster', 'Mes', and 'Tratamiento' and calculate the required metrics
grouped_df = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.groupby([cluster, 'Mes', 'Tratamiento']).agg(
    N_Simulaciones=('Monto_Simulado', 'count'),
    N_Ventas=('venta', 'sum'),
    Tasa_Curse=('venta', lambda x: x.sum() / x.count() if x.count() > 0 else 0),
    Tasa_Promedio=('Tasa_Simulado', 'mean'),
    Monto_Promedio=('Monto_Simulado', 'mean'),
    Plazo_Promedio=('Plazo_Simulado', 'mean'),
    Tasa_Prom=('Tasa_Simulado', 'mean'),
    Revenue_Potencial=('Monto_Simulado', 'sum')
).reset_index()

In [37]:
# Group by 'rut' and calculate the required aggregations
df_grouped = df_simulaciones_e_informacion_de_clientes_ventas.groupby('rut').agg(
    Elasticidad_Precios=('Elasticidad_Precios', 'first'),  # Assuming you want the first value
    Monto_Simulado_Mean=('Monto_Simulado', 'mean'),
    Plazo_Simulado_Mean=('Plazo_Simulado', 'mean'),
    Tasa_Simulado_Mean=('Tasa_Simulado', 'mean'),
    Simulo_Count=('simulo', 'count'),
    Venta_Count=('venta', 'sum')  # Assuming 'venta' is binary (0 or 1)
).reset_index()

df_grouped

,rut,Elasticidad_Precios,Monto_Simulado_Mean,Plazo_Simulado_Mean,Tasa_Simulado_Mean,Simulo_Count,Venta_Count
0,1,Alta,2.685672e+05,30.400000,1.776668,15,5
1,2,Baja,8.333564e+05,22.300000,1.877649,10,2
2,3,Baja,3.167542e+07,28.800000,1.831918,20,4
3,4,Alta,1.401495e+06,20.720000,1.849053,25,9
4,5,Media,4.879516e+05,35.235294,1.772920,17,6
...,...,...,...,...,...,...,...
543646,543647,Baja,8.371279e+05,30.375000,1.913178,8,0
543647,543648,Baja,7.404169e+06,18.444444,1.734614,18,3
543648,543649,Media,1.919464e+06,19.000000,1.842565,14,8
543649,543650,Media,4.353412e+06,18.909091,1.614806,22,12


# 10. CLUSTERING DEFINITIVO POR POLITICAS

### Seteo de cluster

In [38]:
df_informacion_de_clientes_procesados_cluster_definitivo = df_informacion_de_clientes[['rut', 'Categoria_Digital', 'Edad', 'Genero', 'Renta', 'Propension', 'Probabilidad_No_Pago']].copy()


In [39]:
import pandas as pd

# Create a copy of the dataframe to work with
df = df_informacion_de_clientes_procesados_cluster_definitivo.copy()

# Propensión
df['Categoria_Propenso'] = pd.qcut(df['Propension'], 3, labels=['Propension baja', 'Propension media', 'Propension alta'])

# Probabilidad_No_Pago
df['Categoria_Probabilidad_No_Pago'] = pd.cut(df['Probabilidad_No_Pago'], 
                                              bins=[-float('inf'), 0.0011, 0.00149, 0.005, 0.006, float('inf')],
                                              labels=['Muy buen pagador', 'Buen pagador', 'Pagador neutro', 'Mal pagador', 'Muy mal pagador'])

# Edad
df['Categoria_Edad'] = pd.cut(df['Edad'], 
                              bins=[-float('inf'), 35, 60, float('inf')],
                              labels=['Joven', 'Adulto', 'Adulto Mayor'])

# Renta
df_unicos_renta = df[['rut', 'Renta']].drop_duplicates()
df_unicos_renta['Percentil_Renta'] = pd.qcut(df_unicos_renta['Renta'], 100, labels=False) + 1
df_unicos_renta['Categoria_Renta'] = pd.cut(df_unicos_renta['Percentil_Renta'], 
                                            bins=[-float('inf'), 30, 80, float('inf')],
                                            labels=['Renta Baja', 'Renta Media', 'Renta Alta'])

# Merge the renta categories back to the main dataframe
df = df.merge(df_unicos_renta[['rut', 'Categoria_Renta']], on='rut', how='left')

# Display the dataframe with the new columns
df

,rut,Categoria_Digital,Edad,Genero,Renta,Propension,Probabilidad_No_Pago,Categoria_Propenso,Categoria_Probabilidad_No_Pago,Categoria_Edad,Categoria_Renta
0,1,Cliente no Digital,30.0,Masculino,6.258183e+05,0.997340,0.028445,Propension alta,Muy mal pagador,Joven,Renta Media
1,2,Cliente no Digital,41.0,Femenino,3.172616e+05,0.291601,0.014320,Propension baja,Muy mal pagador,Adulto,Renta Baja
2,3,Cliente no Digital,38.0,Femenino,1.240551e+07,0.685085,0.002156,Propension alta,Pagador neutro,Adulto,Renta Alta
3,4,Cliente no Digital,57.0,Masculino,5.441466e+05,0.914672,0.034418,Propension alta,Muy mal pagador,Adulto,Renta Baja
4,5,Cliente Digital,26.0,Masculino,1.870225e+05,0.425077,0.014978,Propension media,Muy mal pagador,Joven,Renta Baja
...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,Cliente Digital,29.0,Femenino,1.176598e+05,0.144657,0.037291,Propension baja,Muy mal pagador,Joven,Renta Baja
543647,543648,Cliente no Digital,31.0,Masculino,1.558612e+06,0.740170,0.035877,Propension alta,Muy mal pagador,Joven,Renta Media
543648,543649,Cliente no Digital,49.0,Masculino,9.449508e+05,0.255285,0.023306,Propension baja,Muy mal pagador,Adulto,Renta Media
543649,543650,Cliente no Digital,40.0,Femenino,1.039964e+06,0.709086,0.015121,Propension alta,Muy mal pagador,Adulto,Renta Media


In [40]:
# Concatenate the specified variables into a new column
df['categoria_clusterizacion'] = (
    df['Categoria_Digital'].astype(str) + ' ' +
    df['Categoria_Edad'].astype(str) + ' de genero ' +
    df['Genero'].astype(str) + ' con ' +
    df['Categoria_Propenso'].astype(str) + ' con una ' +
    df['Categoria_Renta'].astype(str)
)


In [41]:
# Assign a unique number to each unique entry in 'categoria_clusterizacion'
df['categoria_clusterizacion_numerica'] = df['categoria_clusterizacion'].astype('category').cat.codes

In [42]:
asignacion_clusters = df[['rut', 'categoria_clusterizacion', 'categoria_clusterizacion_numerica']].copy()

## 10.2 Estimacion de curvas de elasticidad por cluster

In [43]:
df_estimar_elasticidad = pd.merge(df_simulaciones_e_informacion_de_clientes_ventas, asignacion_clusters, on='rut', how='left')

### Este código calcula y grafica el precio que maximiza el revenue esperado para diferentes clusters en un DataFrame. Utiliza regresión logistica para estimar la probabilidad de aceptación. Luego calcula el revenue potencial y el revenue esperado. Se imprimen los resultados esperados solo por ese precio

In [44]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

lista_revenue = []
lista_clientes = []
lista_creditos = []
lista_simulaciones = []

cluster_results = []  # List to store per-cluster results

# Get unique cluster numbers
cluster_numbers = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()

for cluster_num in cluster_numbers:
    # Filter data for the current cluster
    df_cluster = df_estimar_elasticidad[df_estimar_elasticidad['categoria_clusterizacion_numerica'] == cluster_num]
    
    # Ensure there is data for both venta == 1 and venta == 0
    if df_cluster.empty or df_cluster['venta'].isnull().all():
        continue
    
    # Remove rows where 'venta' or 'Tasa_Simulado' is null or infinite
    df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan)
    df_cluster = df_cluster.dropna(subset=['venta', 'Tasa_Simulado', 'Plazo_Simulado', 'Monto_Simulado', 'Probabilidad_No_Pago'])
    
    # If there are not enough data points, skip the cluster
    if df_cluster.shape[0] < 10:
        continue
    
    # Extract 'venta' and 'Tasa_Simulado'
    y = df_cluster['venta']
    X = df_cluster[['Tasa_Simulado']]
    
    # Add constant term for intercept
    X = sm.add_constant(X)
    
    # Remove rows with NaN or Inf values in X or y
    is_finite = np.isfinite(X).all(1) & np.isfinite(y)
    X = X[is_finite]
    y = y[is_finite]
    
    # Ensure that after removing NaNs/Infs, there is still enough data
    if len(y) < 10:
        continue
    
    # Fit logistic regression model
    logit_model = sm.Logit(y, X)
    try:
        result = logit_model.fit(disp=0)
    except:
        continue  # Skip cluster if model fails to converge
    
    # Create a grid of 'Tasa_Simulado' values
    tasa_min = df_cluster['Tasa_Simulado'].min()
    tasa_max = df_cluster['Tasa_Simulado'].max()
    tasas_grid = np.linspace(tasa_min, tasa_max, 1000)
    
    # Predict acceptance probability
    X_grid = sm.add_constant(tasas_grid)
    acceptance_probability = result.predict(X_grid)
    
    # Ensure probabilities are between 0 and 1
    acceptance_probability = np.clip(acceptance_probability, 0, 1)
    
    # Compute mean values needed for revenue calculations
    n = df_cluster['Plazo_Simulado'].mean()
    vp = df_cluster['Monto_Simulado'].mean()
    pnp = df_cluster['Probabilidad_No_Pago'].mean()
    data = {
        'Plazo_Simulado_medio': n, 
        'Monto_Simulado_medio': vp, 
        'Probabilidad_No_Pago_media': pnp
    }
    
    # Calculate potential revenue
    i = tasas_grid / 100  # Convert to decimal
    one_plus_i_pow_n = np.power(1 + i, n)
    annuity_factor = (i * one_plus_i_pow_n) / (one_plus_i_pow_n - 1)
    revenue = (n * vp * annuity_factor) - vp
    potential_revenue = revenue * (1 - pnp)
    
    # Calculate simulations per date without groupby
    df_cluster_simulaciones_1 = df_cluster[df_cluster['simulo'] == 1]
    num_dates = df_cluster_simulaciones_1['fecha'].nunique()
    total_simulaciones = df_cluster_simulaciones_1['simulo'].sum()
    simulaciones_medias = total_simulaciones / num_dates if num_dates else 0
    
    # Skip if no simulations are available
    if simulaciones_medias == 0:
        continue
    
    # Calculate expected revenue
    expected_revenue = acceptance_probability * potential_revenue * simulaciones_medias
    
    # Find the price that maximizes expected revenue
    idx_max = np.argmax(expected_revenue)
    max_price = tasas_grid[idx_max]
    max_expected_revenue = expected_revenue[idx_max]
    
    # Acceptance probability at optimal price
    prob_aceptacion_optima = acceptance_probability[idx_max]
    
    # Expected number of accepted credits
    num_creditos_aceptados = round(prob_aceptacion_optima * simulaciones_medias)
    
    # Number of unique clients
    num_clients = df_cluster['rut'].nunique()
    
    # Print results
    print(f'Cluster {cluster_num}:')
    print(f'- Precio Máx. Revenue Esperado = {max_price:.2f}%')
    print(f'- Revenue Esperado Máximo = {max_expected_revenue:,.2f}')
    print(f'- Número de clientes en el cluster = {num_clients}')
    print(f'- Número de simulaciones en el cluster = {simulaciones_medias:.2f}')
    print(f'- Probabilidad de aceptación en el precio óptimo = {prob_aceptacion_optima:.4f}')
    print(f'- Número esperado de créditos aceptados = {num_creditos_aceptados}')
    print(f'- Monto medio simulado = {data["Monto_Simulado_medio"]:,.2f}')
    print(f'- Plazo medio simulado = {data["Plazo_Simulado_medio"]:,.2f}')
    print(f'- Probabilidad de no pago media = {data["Probabilidad_No_Pago_media"]:.4f}\n')
    
    # Append to lists
    lista_clientes.append(num_clients)
    lista_revenue.append(max_expected_revenue)
    lista_creditos.append(num_creditos_aceptados)
    lista_simulaciones.append(simulaciones_medias)
    
    # Append per-cluster results to cluster_results
    cluster_results.append({
        'categoria_clusterizacion_numerica': cluster_num,
        'tasa_optima': max_price,
        'probabilidad_aceptacion_optima': prob_aceptacion_optima,
        'revenue_esperado_maximo': max_expected_revenue,
        'numero_clientes': num_clients,
        'numero_simulaciones_medias': simulaciones_medias,
        'numero_creditos_esperados': num_creditos_aceptados,
        'monto_medio_simulado': data["Monto_Simulado_medio"],
        'plazo_medio_simulado': data["Plazo_Simulado_medio"],
        'probabilidad_no_pago_media': data["Probabilidad_No_Pago_media"]
    })

# Print total revenue and other metrics
total_revenue = sum(lista_revenue)
total_clientes = sum(lista_clientes)
total_simulaciones = sum(lista_simulaciones)
total_creditos = sum(lista_creditos)

print(f"El revenue total esperado es: {total_revenue:,.2f} con un total de {total_clientes} clientes, "
      f"{total_simulaciones:,.2f} simulaciones, y {total_creditos} créditos.")

# Create a DataFrame from cluster_results
df_cluster_results = pd.DataFrame(cluster_results)

# Merge the results back to df_estimar_elasticidad
df_estimar_elasticidad = df_estimar_elasticidad.merge(
    df_cluster_results[['categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']],
    on='categoria_clusterizacion_numerica', 
    how='left'
)

Cluster 101:
- Precio Máx. Revenue Esperado = 1.26%
- Revenue Esperado Máximo = 446,196,158.99
- Número de clientes en el cluster = 6422
- Número de simulaciones en el cluster = 1814.08
- Probabilidad de aceptación en el precio óptimo = 0.4998
- Número esperado de créditos aceptados = 907
- Monto medio simulado = 2,672,066.39
- Plazo medio simulado = 27.47
- Probabilidad de no pago media = 0.0276

Cluster 76:
- Precio Máx. Revenue Esperado = 1.22%
- Revenue Esperado Máximo = 65,495,606.51
- Número de clientes en el cluster = 5907
- Número de simulaciones en el cluster = 912.80
- Probabilidad de aceptación en el precio óptimo = 0.5523
- Número esperado de créditos aceptados = 504
- Monto medio simulado = 712,403.67
- Plazo medio simulado = 27.65
- Probabilidad de no pago media = 0.0092

Cluster 72:
- Precio Máx. Revenue Esperado = 1.08%
- Revenue Esperado Máximo = 3,990,650,405.36
- Número de clientes en el cluster = 5431
- Número de simulaciones en el cluster = 1628.33
- Probabilidad d

# 13 Estimacion de respuesta a tratamiento por cluster

In [45]:
df_simulaciones_info = pd.merge(df_tratamiento, df_simulaciones_clientes, on=['rut', 'fecha'], how='left')

In [46]:
# Use vectorized string operations to create the 'Tratamiento' column
df_simulaciones_info['Tratamiento'] = (
    'Ejecutivo=' + df_simulaciones_info['asg_ejec'].astype(str) +
    ', Correos=' + df_simulaciones_info['n_correos'].astype(int).astype(str)
)

In [47]:
df_simulaciones_info['mes'] = pd.to_datetime(df_simulaciones_info['fecha']).dt.to_period('M')

df_simulaciones_info[df_simulaciones_info['rut'] == 1]

,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05
...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05


In [48]:

df_simulaciones_info['simulo'] = df_simulaciones_info['Unnamed: 0_y'].notna().astype(int)
df_simulaciones_info[df_simulaciones_info['rut'] == 1]

,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes,simulo
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01,0
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02,0
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03,0
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04,0
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02,0
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03,0
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04,0
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05,1


In [49]:
df1 = df_estimar_elasticidad[['rut', 'categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()
df2 = df_simulaciones_info[['rut', 'mes', 'Tratamiento', 'simulo']].copy()

In [50]:
import pandas as pd

# Paso 1: Preparación de datos y mapeo de clusters
df1_unique = df1.drop_duplicates(subset='rut')
rut_cluster_map = df1_unique.set_index('rut')['categoria_clusterizacion_numerica']
df2['categoria_clusterizacion_numerica'] = df2['rut'].map(rut_cluster_map)
df2 = df2.dropna(subset=['categoria_clusterizacion_numerica'])

# Conversión de tipos de datos
df2['categoria_clusterizacion_numerica'] = df2['categoria_clusterizacion_numerica'].astype(int)
df2['simulo'] = pd.to_numeric(df2['simulo'], errors='coerce').fillna(0).astype(int)
df2['Tratamiento'] = df2['Tratamiento'].astype('category')

# Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
total_entries_per_cluster_treatment = df2.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

# Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
df_simulations = df2[df2['simulo'] == 1]
favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

# Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
# Hacer un merge de total_entries_per_cluster_treatment y favorable_entries_per_cluster_treatment
df_probabilities = total_entries_per_cluster_treatment.merge(
    favorable_entries_per_cluster_treatment,
    on=['categoria_clusterizacion_numerica', 'Tratamiento'],
    how='left'
)

# Llenar valores nulos en 'caso_favorable' con 0 en las columnas numéricas solamente
df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

# Asegurar que los conteos sean enteros para 'caso_total'
df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

# Calcular la probabilidad de simulación
df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

# Organizar los resultados
df_probabilities = df_probabilities[[
    'categoria_clusterizacion_numerica',
    'Tratamiento',
    'probabilidad_simular',
    'caso_favorable',
    'caso_total'
]]

# Mostrar el DataFrame resultante
df_probabilities

,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
0,0,"Ejecutivo=0, Correos=0",0.238938,27,113
1,0,"Ejecutivo=0, Correos=1",0.201613,25,124
2,0,"Ejecutivo=0, Correos=2",0.222222,18,81
3,0,"Ejecutivo=0, Correos=3",0.247104,64,259
4,0,"Ejecutivo=0, Correos=4",0.250000,96,384
...,...,...,...,...,...
859,107,"Ejecutivo=0, Correos=3",0.152493,10569,69308
860,107,"Ejecutivo=0, Correos=4",0.152522,16988,111381
861,107,"Ejecutivo=1, Correos=0",0.341773,6007,17576
862,107,"Ejecutivo=1, Correos=1",0.349486,6664,19068


In [51]:
cluster = 50

In [52]:
df_probabilities[df_probabilities['categoria_clusterizacion_numerica'] == cluster].sort_values('probabilidad_simular', ascending=False)


,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
407,50,"Ejecutivo=1, Correos=2",0.319331,57485,180017
406,50,"Ejecutivo=1, Correos=1",0.298788,9341,31263
405,50,"Ejecutivo=1, Correos=0",0.279709,8118,29023
404,50,"Ejecutivo=0, Correos=4",0.133107,24163,181530
403,50,"Ejecutivo=0, Correos=3",0.128865,14625,113491
402,50,"Ejecutivo=0, Correos=2",0.119665,5437,45435
401,50,"Ejecutivo=0, Correos=1",0.100729,5981,59377
400,50,"Ejecutivo=0, Correos=0",0.078941,4319,54712


In [53]:
# Calculate the mean probability by 'Tratamiento'
mean_probability_by_treatment = df_probabilities.groupby('Tratamiento')['probabilidad_simular'].mean()

# Display the result
mean_probability_by_treatment.sort_values(ascending=False)


Tratamiento
Ejecutivo=1, Correos=2    0.366290
Ejecutivo=1, Correos=1    0.359382
Ejecutivo=1, Correos=0    0.341124
Ejecutivo=0, Correos=4    0.176108
Ejecutivo=0, Correos=3    0.173983
Ejecutivo=0, Correos=2    0.166333
Ejecutivo=0, Correos=1    0.152964
Ejecutivo=0, Correos=0    0.145320
Name: probabilidad_simular, dtype: float64

In [54]:
df_estimar_elasticidad

df_estimar_elasticidad['Monto_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Monto_Simulado'].transform('mean')
df_estimar_elasticidad['Plazo_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('mean')

In [55]:
# Select only necessary columns from each DataFrame
df_estimar_elasticidad_small = df_estimar_elasticidad[['categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago', 'Monto_Simulado_mean', 'Plazo_Simulado_mean']]
df_probabilities_small = df_probabilities[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']]

# Merge the reduced DataFrames
df_asignacion_de_tratamientos = pd.merge(df_estimar_elasticidad_small, df_probabilities_small, on='categoria_clusterizacion_numerica', how='left')


In [56]:
import os
from datetime import datetime

# Create a timestamped folder name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"cluster_data_{timestamp}"
os.makedirs(folder_name, exist_ok=True)

# Save Cluster Info
df_cluster_info = df_estimar_elasticidad_small[['categoria_clusterizacion_numerica', 'Monto_Simulado_mean', 'Plazo_Simulado_mean', 'probabilidad_aceptacion_optima', 'tasa_optima']].drop_duplicates()
df_cluster_info.to_csv(os.path.join(folder_name, 'cluster_info.csv'), index=False)

# Save Probabilities and Treatment
df_probabilities_treatment = df_probabilities_small[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']].drop_duplicates()
df_probabilities_treatment.to_csv(os.path.join(folder_name, 'probabilities_treatment.csv'), index=False)

# Save RUT Info
df_rut_info = df_estimar_elasticidad_small[['rut', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago']].drop_duplicates()
df_rut_info.to_csv(os.path.join(folder_name, 'rut_info.csv'), index=False)

print(f"Files saved in folder: {folder_name}")



Files saved in folder: cluster_data_20241106_020021


# 14. Modelo de asignacion

In [1]:
import pandas as pd
import os

# Definir la carpeta base
base_folder = "cluster_data_20241106_020021/"

# Función para cargar el archivo y mapear tratamientos
def cargar_y_mapear_tratamientos(file_path, tratamiento_map):
    df = pd.read_csv(file_path)
    df['tratamiento_id'] = df['Tratamiento'].map(tratamiento_map)
    df.to_csv(file_path, index=False)
    print("Columna 'tratamiento_id' añadida con éxito.")
    return df

# Función para añadir la columna 'tratamientos' con [probabilidad_simular, tratamiento_id]
def crear_lista_tratamientos(df_prob, df_rut):
    df_prob['tratamientos'] = df_prob[['probabilidad_simular', 'tratamiento_id']].values.tolist()
    grouped_prob = df_prob.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()
    
    if 'tratamientos' in df_rut.columns:
        df_rut.drop(columns='tratamientos', inplace=True)
        
    df_rut = df_rut.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')
    return df_rut

# Función para unir df_rut_info con información de clusters
def unir_cluster_info(df_rut, df_cluster, output_path):
    df_rut = df_rut.merge(df_cluster, on='categoria_clusterizacion_numerica', how='left')
    df_rut.to_csv(output_path, index=False)
    print("Variables de cluster_info añadidas a rut_info.csv con éxito.")
    return df_rut

# Función para ajustar la 'tasa_optima' y calcular 'RC'
def calcular_RC(df_rut, output_path):
    df_rut['tasa_optima'] = df_rut['tasa_optima'] / 100
    df_rut['RC'] = (
        (df_rut['Plazo_Simulado_mean'] * df_rut['Monto_Simulado_mean'] * df_rut['tasa_optima'] *
         ((1 + df_rut['tasa_optima']) ** df_rut['Plazo_Simulado_mean'])) /
        (((1 + df_rut['tasa_optima']) ** df_rut['Plazo_Simulado_mean']) - 1)
    ) - df_rut['Monto_Simulado_mean']
    
    df_rut.to_csv(output_path, index=False)
    print("La variable 'tasa_optima' fue convertida y 'RC' fue calculada y añadida a rut_info.csv.")
    return df_rut

# Mapear tratamientos y cargar el archivo inicial
tratamiento_map = {
    "Ejecutivo=0, Correos=0": 1,
    "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3,
    "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5,
    "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7,
    "Ejecutivo=1, Correos=2": 8
}

# Ruta de los archivos en la carpeta base
probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
rut_info_file = os.path.join(base_folder, "rut_info.csv")
cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# Paso 1: Cargar y mapear tratamientos
df_probabilities = cargar_y_mapear_tratamientos(probabilities_file, tratamiento_map)

# Paso 2: Crear lista de tratamientos y unir con rut_info
df_rut_info = pd.read_csv(rut_info_file)
df_rut_info = crear_lista_tratamientos(df_probabilities, df_rut_info)
df_rut_info.to_csv(rut_info_file, index=False)
print("Variable 'tratamientos' añadida a rut_info.csv.")

# Paso 3: Unir rut_info con cluster_info
df_cluster_info = pd.read_csv(cluster_info_file)
df_rut_info = unir_cluster_info(df_rut_info, df_cluster_info, rut_info_file)

# Paso 4: Ajustar 'tasa_optima' y calcular 'RC'
df_rut_info = calcular_RC(df_rut_info, rut_info_file)

Columna 'tratamiento_id' añadida con éxito.
Variable 'tratamientos' añadida a rut_info.csv.
Variables de cluster_info añadidas a rut_info.csv con éxito.
La variable 'tasa_optima' fue convertida y 'RC' fue calculada y añadida a rut_info.csv.


In [8]:
import pandas as pd
import ast
from gurobipy import Model, GRB, quicksum

# Load data and take a sample subset
file_path = "cluster_data_20241106_020021/rut_info.csv"
print("Loading data and selecting a subset...")
datos = pd.read_csv(file_path)

# Select a subset (e.g., 1% of the data)
subset_fraction = 1  # Adjust this fraction as desired
datos = datos.sample(frac=subset_fraction, random_state=42)  # Set a random seed for reproducibility
datos = datos.reset_index(drop=True)  # Reset index to avoid KeyError
print(f"Subset selected: {len(datos)} rows")

# Continue with data processing as usual
print("Processing 'tratamientos' column...")
datos["tratamientos"] = datos["tratamientos"].apply(ast.literal_eval)

# Convert relevant columns to numeric
for col in ["RC", "Probabilidad_No_Pago", "probabilidad_aceptacion_optima"]:
    datos[col] = pd.to_numeric(datos[col], errors="coerce")

capacidad_ejecutivos = 205000

# Parameters
costosms = 100
capacidad_ejecutivos = int(capacidad_ejecutivos * subset_fraction)  # Scale capacity for subset

# Set up the optimization model
print("Setting up optimization model...")
model = Model("Maximizar_Ganancias")
model.ModelSense = GRB.MAXIMIZE

# Pre-calculate profits
print("Pre-calculating profits...")
datos['precalc'] = datos.apply(lambda row: [
    row["RC"] * (1 - row["Probabilidad_No_Pago"]) *
    row["probabilidad_aceptacion_optima"] * row["tratamientos"][t - 1][0] - 
    row["tratamientos"][t - 1][1] * costosms
    for t in range(1, 9)
], axis=1)

# Create decision variables only for profitable treatments
variables = {
    i: {
        t: model.addVar(vtype=GRB.BINARY, name=f"tratamiento_{t}_{i}")
        for t in range(1, 9) if datos["precalc"][i][t - 1] > 0
    }
    for i in datos.index
}

# Set the objective function
model.setObjective(quicksum(variables[i][t] * datos["precalc"][i][t - 1]
                            for i in datos.index for t in variables[i]))

# Constraints
# Constraint 1: Only one treatment per client
for i in datos.index:
    model.addConstr(quicksum(variables[i].values()) == 1, f"OneTreatmentPerClient_{i}")

# Constraint 2: Capacity of executives
model.addConstr(quicksum(variables[i][t] for i in datos.index for t in [6, 7, 8] if t in variables[i]) <= capacidad_ejecutivos,
                "CapacityConstraint")

# Constraint 3: Cluster consistency
clusters = datos.groupby("categoria_clusterizacion_numerica").indices
for cluster_id, indices_cluster in clusters.items():
    indices_cluster = list(indices_cluster)  # Convert index to list for consistency
    leader_index = indices_cluster[0]
    for t in range(1, 9):
        if t in variables[leader_index]:
            leader_var = variables[leader_index][t]
            for i in indices_cluster[1:]:
                if t in variables[i]:
                    model.addConstr(variables[i][t] == leader_var, f"ClusterConsistency_{cluster_id}_{t}")

# Optimize the model
print("Solving the problem (this may take a while on larger subsets)...")
model.optimize()

# Extract results
resultados_por_cluster = {}
for cluster_id, indices_cluster in clusters.items():
    leader_index = list(indices_cluster)[0]
    for t in variables[leader_index]:
        if variables[leader_index][t].X == 1:
            resultados_por_cluster[cluster_id] = t
            break

# Calculate total profits
ganancias_totales = model.ObjVal

# Print results
print("Tratamientos asignados por cluster:")
for cluster_id, tratamiento in resultados_por_cluster.items():
    print(f"Cluster {cluster_id}: Tratamiento {tratamiento}")

print(f"\nGanancias totales: {ganancias_totales}")
print("Optimization complete.")


Loading data and selecting a subset...
Subset selected: 543651 rows
Processing 'tratamientos' column...
Setting up optimization model...
Pre-calculating profits...
Solving the problem (this may take a while on larger subsets)...
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 2500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4891996 rows, 4349208 columns and 14676849 nonzeros
Model fingerprint: 0xeca36958
Variable types: 0 continuous, 4349208 integer (4349208 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+03, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 0 rows and 0 columns (presolve time = 5s) ...
Presolve removed 4348344 rows and 950000 columns (presolve time = 10s) ...
Presolve removed 4348344 rows and 4348344 columns (preso

In [13]:
from collections import Counter

for cluster_id, tratamiento in resultados_por_cluster.items():
    # Count the number of times each treatment is assigned
    treatment_counts = Counter(resultados_por_cluster.values())

# Print the counts for each treatment
for treatment, count in treatment_counts.items():
    print(f"Treatment {treatment}: {count} times")

Treatment 8: 39 times
Treatment 7: 14 times
Treatment 6: 3 times
Treatment 5: 32 times
Treatment 4: 9 times
Treatment 3: 10 times
Treatment 1: 1 times
